In [ ]:
import torch, torchaudio, torchvision.transforms as transforms, matplotlib.pyplot as plt, torch.nn as nn, torch.optim as optim, numpy as np
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import  StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, auc, classification_report, roc_auc_score
from torch.autograd import grad


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

data = np.load("../hvcm/RFQ.npy", allow_pickle=True)
label = np.load("../hvcm/RFQ_labels.npy", allow_pickle=True)
label = label[:, 1]  # Assuming the second column is the label
label = (label == "Fault").astype(int)  # Convert to binary labels
print(data.shape, label.shape)

normal_indices = np.where(label == 0)

# Wasserstein GAN

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.init_size = (14, 2, 2, 2)  # Initial shape before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, int(torch.prod(torch.tensor(self.init_size)))))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm3d(14),
            nn.Upsample(scale_factor=2),  # (N, 14, 4, 4, 4)
            nn.Conv3d(14, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(128, 0.8),
            nn.ReLU(inplace=True),

            nn.Upsample(scale_factor=2),  # (N, 128, 8, 8, 8)
            nn.Conv3d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(64, 0.8),
            nn.ReLU(inplace=True),

            nn.Upsample(scale_factor=(2, 2, 2)),  # (N, 64, 16, 16, 16)
            nn.Conv3d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(32, 0.8),
            nn.ReLU(inplace=True),

            nn.Conv3d(32, 14, kernel_size=3, stride=1, padding=1),
            nn.Upsample(size=(10, 15, 30)),  # Final shape
            # nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], *self.init_size)
        return self.conv_blocks(out)



# Discriminator (Critic)
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv3d(14, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv3d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv3d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm3d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.AdaptiveAvgPool3d((1, 1, 1))  # Ensures fixed output size
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 1)  # 512 channels after AdaptiveAvgPool3d
        )

    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)


# Gradient Penalty
def compute_gradient_penalty(discriminator, real_samples, fake_samples):
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, 1).to(device)
    interpolates = (alpha * real_samples + (1 - alpha) * fake_samples).requires_grad_(True)
    d_interpolates = discriminator(interpolates)
    fake = torch.ones_like(d_interpolates).to(device)
    gradients = grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]
    gradients = gradients.reshape(gradients.size(0), -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

# Initialize models
latent_dim = 100
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=1e-4, betas=(0.0, 0.9))
optimizer_D = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.0, 0.9))

# Training parameters
n_epochs = 100
batch_size = 64
lambda_gp = 20
n_critic = 3

# Training loop
for epoch in range(n_epochs):
    for i in range(0, len(normal_indices[0]), batch_size):
        real_data_np = data[normal_indices[i:i+batch_size]]
        real_data_tensor = torch.tensor(real_data_np, dtype=torch.float32)

        real_data = real_data_tensor.reshape(-1, 4500, 14).permute(0, 2, 1)  # (690, 14, 4500)
        real_data = real_data.reshape(-1, 14, 10, 15, 30).to(device)         # (690, 14, 10, 15, 30)
       

        # Train Discriminator
        for _ in range(n_critic):
            z = torch.randn(real_data.size(0), latent_dim).to(device)
            fake_data = generator(z).detach()
            d_real = discriminator(real_data)
            d_fake = discriminator(fake_data)
            gp = compute_gradient_penalty(discriminator, real_data, fake_data)
            loss_D = -torch.mean(d_real) + torch.mean(d_fake) + lambda_gp * gp

            optimizer_D.zero_grad()
            loss_D.backward()
            optimizer_D.step()

        # Train Generator
        z = torch.randn(real_data.size(0), latent_dim).to(device)
        fake_data = generator(z)
        loss_G = -torch.mean(discriminator(fake_data))

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

    print(f"[Epoch {epoch+1}/{n_epochs}] Loss D: {loss_D.item():.4f}, Loss G: {loss_G.item():.4f}, GP: {gp.item():.4f}")

    # Optional: Save model checkpoints
    if (epoch + 1) % 10 == 0:
        torch.save(generator.state_dict(), f"generator_epoch_{epoch+1}.pth")
        torch.save(discriminator.state_dict(), f"discriminator_epoch_{epoch+1}.pth")

# Generate and Combine

In [ ]:
generator = Generator(latent_dim).to(device)
generator.load_state_dict(torch.load("generator_epoch_30.pth")) # Use weights from the last epoch
num_samples = len(data[normal_indices]) # or however many samples you want
z = torch.randn(num_samples, 100).to(device)
generated_samples = generator(z).detach().cpu().numpy()
generated_samples = generated_samples.reshape(num_samples, 14, 4500).transpose(0, 2, 1)  # (num_samples, 4500, 14)
combine_data = np.concatenate((generated_samples, data), axis=0)  # Combine real and generated data
combine_labels = np.concatenate((np.zeros(num_samples), label), axis=0)  # Labels: 0 for real, 0 for generated

# Processing: Mel Spec > Resizing > Feature Extraction

In [ ]:
# Resize and convert to 3-channel image
def resize_spectrogram(spectrogram):
    spectrogram = (spectrogram - spectrogram.min()) / (spectrogram.max() - spectrogram.min() + 1e-6)
    spectrogram = np.uint8(spectrogram.cpu().numpy() * 255)
    spectrogram = np.stack([spectrogram] * 3, axis=-1)
    image = Image.fromarray(spectrogram)
    image = transforms.Resize((224, 224))(image)
    return transforms.ToTensor()(image)

# Process dataset
def process_dataset(data):
    num_samples, _, num_channels = data.shape
    features = np.zeros((num_samples, num_channels, 4096))
    mel_transform = torchaudio.transforms.MelSpectrogram(sample_rate=2500000, n_mels=128).to(device)
    model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1).to(device)
    model.classifier = model.classifier[:-3]
    model.eval()

    for i in range(num_samples):
        for j in range(num_channels):
            ts = torch.tensor(data[i, :, j], dtype=torch.float32).to(device)
            mel = mel_transform(ts)
            img = resize_spectrogram(mel)
            with torch.no_grad():
                feat = model(img.unsqueeze(0).to(device))
            features[i, j, :] = feat.squeeze().cpu().numpy()
    return features

# AE Class

In [ ]:
# Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self, input_size=4096):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 64), 
            nn.ReLU(),
            nn.Linear(64, 32), 
            nn.ReLU(),
            nn.Linear(32, 16), 
            nn.ReLU(),
            nn.Linear(16, 8), 
            nn.ReLU(),
            nn.Linear(8, 4), 
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 16), 
            nn.ReLU(),
            nn.Linear(16, 32), 
            nn.ReLU(),
            nn.Linear(32, 64), 
            nn.ReLU(),
            nn.Linear(64, input_size), 
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))



# Train autoencoder
def train_autoencoder(features, epochs=20, batch_size=128):
    x = torch.tensor(features.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader = DataLoader(TensorDataset(x), batch_size=batch_size, shuffle=True)
    model = Autoencoder().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        total_loss = 0
        for batch in loader:
            inputs = batch[0]
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(loader):.6f}")
    return model


def print_eval(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))
  print("F1 = {}".format(f1_score(labels, predictions)))
  print(confusion_matrix(labels, predictions))

# Plot reconstruction error histogram
def plot_reconstruction_error(model, features, percentile=95):
    x = torch.tensor(features.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader = DataLoader(TensorDataset(x), batch_size=64)
    errors = []
    criterion = nn.MSELoss(reduction='none')

    with torch.no_grad():
        for batch in loader:
            inputs = batch[0]
            outputs = model(inputs)
            batch_errors = criterion(outputs, inputs).mean(dim=1)
            errors.extend(batch_errors.cpu().numpy())

    threshold = np.percentile(errors, percentile)
    anomalies = np.sum(np.array(errors) > threshold)

    plt.hist(errors, bins=50, alpha=0.75)
    plt.axvline(threshold, color='r', linestyle='--', label=f'Threshold ({percentile}%)')
    plt.xlabel('Reconstruction Error')
    plt.ylabel('Frequency')
    plt.title('Reconstruction Error Histogram')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"Anomaly threshold: {threshold:.6f}")
    print(f"Detected anomalies: {anomalies}")


# Cross Validation without Scalers

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
features = process_dataset(combine_data)
normal_indices = np.where(combine_labels == 0)[0]
print("Features shape:", features.shape)
for fold, (train_idx, val_idx) in enumerate(skf.split(features, combine_labels)):
    print(f"Fold {fold + 1}")
    train_fold_data, val_fold_data = features[train_idx], features[val_idx]
    train_fold_labels, val_fold_labels = combine_labels[train_idx], combine_labels[val_idx]

    # Train autoencoder on the training fold
    model = train_autoencoder(features[normal_indices], epochs=15, batch_size=64)

    # Evaluate on validation fold
    x_val = torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader_val = DataLoader(TensorDataset(x_val), batch_size=64)
    
    # Compute reconstruction errors
    x = model(torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)).cpu().detach().numpy()
    errors = np.mean((x - val_fold_data.reshape(-1, 4096)) ** 2, axis=1)

    # Reshape to (175, 14)
    errors = errors.reshape(val_fold_data.shape[0], val_fold_data.shape[1])

    # Aggregate per sample (e.g., mean across channels)
    sample_errors = np.mean(errors, axis=1)

    percentile = 90
    # Thresholding
    threshold = np.percentile(sample_errors, percentile)
    predictions = (sample_errors > threshold).astype(int)


    
    plot_reconstruction_error(model, val_fold_data, percentile=percentile)
    print_eval(predictions, val_fold_labels)

# Cross Validation with StandardScaler

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scaled_data = StandardScaler().fit_transform(combine_data.reshape(-1, data.shape[-1])).reshape(combine_data.shape)
features = process_dataset(scaled_data)
normal_indices = np.where(combine_labels == 0)[0]
print("Features shape:", features.shape)
for fold, (train_idx, val_idx) in enumerate(skf.split(features, combine_labels)):
    print(f"Fold {fold + 1}")
    train_fold_data, val_fold_data = features[train_idx], features[val_idx]
    train_fold_labels, val_fold_labels = combine_labels[train_idx], combine_labels[val_idx]

    # Train autoencoder on the training fold
    model = train_autoencoder(features[normal_indices], epochs=15, batch_size=64)

    # Evaluate on validation fold
    x_val = torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader_val = DataLoader(TensorDataset(x_val), batch_size=64)
    
    # Compute reconstruction errors
    x = model(torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)).cpu().detach().numpy()
    errors = np.mean((x - val_fold_data.reshape(-1, 4096)) ** 2, axis=1)

    # Reshape to (175, 14)
    errors = errors.reshape(val_fold_data.shape[0], val_fold_data.shape[1])

    # Aggregate per sample (e.g., mean across channels)
    sample_errors = np.mean(errors, axis=1)

    percentile = 90
    # Thresholding
    threshold = np.percentile(sample_errors, percentile)
    predictions = (sample_errors > threshold).astype(int)


    
    plot_reconstruction_error(model, val_fold_data, percentile=percentile)
    print_eval(predictions, val_fold_labels)

# Cross Validation with MinMax

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scaled_data = MinMaxScaler().fit_transform(combine_data.reshape(-1, combine_data.shape[-1])).reshape(combine_data.shape)
features = process_dataset(scaled_data)
normal_indices = np.where(combine_labels == 0)[0]
print("Features shape:", features.shape)
for fold, (train_idx, val_idx) in enumerate(skf.split(features, combine_labels)):
    print(f"Fold {fold + 1}")
    train_fold_data, val_fold_data = features[train_idx], features[val_idx]
    train_fold_labels, val_fold_labels = combine_labels[train_idx], combine_labels[val_idx]

    # Train autoencoder on the training fold
    model = train_autoencoder(features[normal_indices], epochs=15, batch_size=64)

    # Evaluate on validation fold
    x_val = torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)
    loader_val = DataLoader(TensorDataset(x_val), batch_size=64)
    
    # Compute reconstruction errors
    x = model(torch.tensor(val_fold_data.reshape(-1, 4096), dtype=torch.float32).to(device)).cpu().detach().numpy()
    errors = np.mean((x - val_fold_data.reshape(-1, 4096)) ** 2, axis=1)

    # Reshape to (175, 14)
    errors = errors.reshape(val_fold_data.shape[0], val_fold_data.shape[1])

    # Aggregate per sample (e.g., mean across channels)
    sample_errors = np.mean(errors, axis=1)

    percentile = 90
    # Thresholding
    threshold = np.percentile(sample_errors, percentile)
    predictions = (sample_errors > threshold).astype(int)


    
    plot_reconstruction_error(model, val_fold_data, percentile=percentile)
    print_eval(predictions, val_fold_labels)

# Observation:
Comparing with and without normalizing data 

### MinMaxed scored

Accuracy = 0.7126436781609196

Precision = 0.1111111111111111

Recall = 0.05555555555555555

F1 = 0.07407407407407407

[[122  16]

[ 34   2]]

---

### StandardScaled scored


Accuracy = 0.896551724137931

Precision = 1.0

Recall = 0.5

F1 = 0.6666666666666666

[[138   0]

[ 18  18]]

---

### Without any normlaization scored (highest in cross-val):

Accuracy = 0.896551724137931

Precision = 1.0

Recall = 0.5

F1 = 0.6666666666666666

[[138   0]

[ 18  18]]